# Data Exploration: Combining all datasets together

In [1]:
import os
import requests
import configparser
config = configparser.ConfigParser()
config.read(os.path.join(os.path.dirname(os.getcwd()), 'config.ini'))

['/home/lpascual/Projects/PoliceShootingsDashboard/config.ini']

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, BooleanType, FloatType

spark = SparkSession.builder.master('local[*]').appName('DataExploration-Combine').getOrCreate()

In [3]:
# Schema of the Police Shootings Dataset
psSchema = StructType([\
                       StructField('id', IntegerType(), False),
                       StructField('date', DateType(), True),
                       StructField('threat_level', StringType(), True),
                       StructField('flee', StringType(), True),
                       StructField('armed', StringType(), True),
                       StructField('city', StringType(), True),
                       StructField('county', StringType(), True),
                       StructField('state', StringType(), True),
                       StructField('longitude', FloatType(), True),
                       StructField('latitude', FloatType(), True),
                       StructField('location_precision', StringType(), True),
                       StructField('name', StringType(), True),
                       StructField('age', IntegerType(), True),
                       StructField('gender', StringType(), True),
                       StructField('race', StringType(), True),
                       StructField('race_source', StringType(), True),
                       StructField('mental_illness', BooleanType(), True),
                       StructField('body_camera', BooleanType(), True),
                       StructField('agency_ids', IntegerType(), True),
                        ])

psaSchema = StructType([\
                       StructField('id', IntegerType(), False),
                       StructField('name', StringType(), True),
                       StructField('type', StringType(), True),
                       StructField('state', StringType(), True),
                       StructField('oricodes', StringType(), True),
                       StructField('total_shootings', IntegerType(), True)
                        ])  

headersAuth = {
    'Authorization': 'Bearer '+ config['unemploymentAPI']['unemployment_api_key']
}

In [4]:
# US Cities and Counties
usCitiesDF = spark.read.option('header', 'True').option('inferSchema', 'true').csv(config['pathways']['us_cities'])
usCitiesDF.createOrReplaceTempView('usCities')

# US Demographics
usDemoDF = spark.read.option('header', 'true').option('inferSchema', 'true').csv(config['pathways']['us_demographics'])
usDemoDF.createOrReplaceTempView('usDemo')

# US Unemployment by County
endPointTemplate = 'https://api.careeronestop.org/v1/unemployment/{}/{}/{}'
url = endPointTemplate.format(config['unemploymentAPI']['unemployment_userID'], 'CA', 'county')
response = requests.get(url, headers=headersAuth, verify=True)
response = response.json()['CountyList']
caUEDataRDD = spark.sparkContext.parallelize(response)
caUEDataDF = spark.read.json(caUEDataRDD)
caUEDataDF = caUEDataDF.withColumn("State", func.lit('CA'))
caUEDataDF.createOrReplaceTempView('usUnemployment')

# Police Shootings
psDF = spark.read.option('header', 'True').schema(psSchema).csv(config['pathways']['police_shootings'])
psDF.createOrReplaceTempView('policeShootings')

psaDF = spark.read.option('header', 'True').schema(psaSchema).csv(config['pathways']['police_shootings_agencies'])
psaDF.createOrReplaceTempView('policeShootingsAgencies')

## US Cities and Counties


In [5]:
usCitiesDF = spark.sql("""
    SELECT 
        state_id,
        state_name,
        county_name as county,
        city
    FROM usCities
""")

usCitiesDF.createOrReplaceTempView('usCitiesNorm')
usCitiesDF.show(5)

+--------+----------+-----------+-----------+
|state_id|state_name|     county|       city|
+--------+----------+-----------+-----------+
|      NY|  New York|   New York|   New York|
|      CA|California|Los Angeles|Los Angeles|
|      IL|  Illinois|       Cook|    Chicago|
|      FL|   Florida| Miami-Dade|      Miami|
|      TX|     Texas|     Dallas|     Dallas|
+--------+----------+-----------+-----------+
only showing top 5 rows



## US Demographics

In [6]:
# Normalize and Select relative columns
# Note: Records that have null values in the following columns: sex, race, min_age, max_age
#       need to be filtered out. Otherwise overcounting occurs.
#       Null in those columns, represents (Across all {sex, race, age})
usDemoNorm = spark.sql("""   
                        SELECT
                            state_name,
                            county_name,
                            sex,
                            min_age,
                            max_age,
                            year,
                            CASE 
                                WHEN race like 'AMERICAN INDIAN%' then 'American Indian'
                                WHEN race like 'SOME OTHER RACE%' then 'Other'
                                WHEN race like 'WHITE%' then 'White'
                                WHEN race like 'ASIAN%' then 'Asian'
                                WHEN race like 'NATIVE HAWAIIAN%' then 'Native Hawaiian'
                                WHEN race like 'TWO OR MORE%' then 'Mixed'
                                WHEN race like 'BLACK%' then 'African American'
                            END as race,
                            population
                        FROM usDemo
                        WHERE year = '2010' AND race is NOT NULL AND sex is NOT NULL 
                              AND min_age is NOT NULL AND max_age is NOT NULL
                        """)
usDemoNorm.createOrReplaceTempView('usDemoNorm')
usDemoNorm.show(5)

+----------+---------------+------+-------+-------+----+---------------+----------+
|state_name|    county_name|   sex|min_age|max_age|year|           race|population|
+----------+---------------+------+-------+-------+----+---------------+----------+
|     Texas|Crockett County|  Male|    5.0|    9.0|2010|          White|       123|
|     Texas|Crockett County|Female|   67.0|   69.0|2010|          Asian|         1|
|     Texas|Crockett County|Female|   55.0|   59.0|2010|          White|       111|
|     Texas|Crockett County|  Male|   21.0|   21.0|2010|          White|         7|
|     Texas|Crockett County|Female|   67.0|   69.0|2010|Native Hawaiian|         0|
+----------+---------------+------+-------+-------+----+---------------+----------+
only showing top 5 rows



## Police Shootings

In [7]:
policeShootingsNorm = spark.sql("""
        SELECT 
            id,
            name,
            date,
            armed,
            age,
            gender,
            mental_illness,
            threat_level,
            flee,
            body_camera,
            CASE
                WHEN race = 'A' THEN 'Asian'
                WHEN race = 'B' THEN 'Black'
                WHEN race = 'N' THEN 'Native'
                WHEN race = 'H' THEN 'Hispanic'
                WHEN race = 'W' THEN 'White'
                WHEN race = 'O' THEN 'Other'
                WHEN race = 'B;H' THEN 'Black and Hispanic'
                ELSE 'Not Documented'
            END as race, 
            city, 
            state as state_id,
            agency_ids
        FROM    
          policeShootings

""")
policeShootingsNorm.createOrReplaceTempView('policeShootingsNorm')
policeShootingsNorm.show(5)

+---+------------------+----------+-------+---+------+--------------+------------+----+-----------+--------+-------------+--------+----------+
| id|              name|      date|  armed|age|gender|mental_illness|threat_level|flee|body_camera|    race|         city|state_id|agency_ids|
+---+------------------+----------+-------+---+------+--------------+------------+----+-----------+--------+-------------+--------+----------+
|  3|        Tim Elliot|2015-01-02|    gun| 53|  male|          true|       point| not|      false|   Asian|      Shelton|      WA|        73|
|  4|  Lewis Lee Lembke|2015-01-02|    gun| 47|  male|         false|       point| not|      false|   White|        Aloha|      OR|        70|
|  5|John Paul Quintero|2015-01-03|unarmed| 23|  male|         false|        move| not|      false|Hispanic|      Wichita|      KS|       238|
|  8|   Matthew Hoffman|2015-01-04|replica| 32|  male|          true|       point| not|      false|   White|San Francisco|      CA|       196|

## US Unemployment

In [8]:
caUEDataDF.createOrReplaceTempView("caUE")
caUEDataDF = spark.sql("""
        SELECT 
            State as state_id, 
            REPLACE(AreaName, ' County', '') as county,
            UnEmpCount as unemployment_count,
            UnEmpRate as unemployment_rate
        FROM caUE
        """)

caUEDataDF.createOrReplaceTempView("caUE")
caUEDataDF.show(5)

+--------+---------+------------------+-----------------+
|state_id|   county|unemployment_count|unemployment_rate|
+--------+---------+------------------+-----------------+
|      CA|  Alameda|             31417|              3.8|
|      CA|   Alpine|                33|              5.3|
|      CA|   Amador|               839|              5.8|
|      CA|    Butte|              5325|              5.7|
|      CA|Calaveras|              1041|              4.6|
+--------+---------+------------------+-----------------+
only showing top 5 rows



## California Subset

In [9]:
spark.sql("""
    SELECT 
        usc.state_name,
        usc.county,
        ps.name, 
        ps.armed,
        ps.age,
        ps.gender,
        ps.mental_illness,
        ps.threat_level,
        ps.flee,
        ps.body_camera,
        ps.race,
        ue.unemployment_count,
        ue.unemployment_rate,
        psa.name as agency_name,
        psa.type as agency_dept
    FROM policeShootingsNorm as ps
    JOIN policeShootingsAgencies as psa
    ON ps.agency_ids = psa.id
    JOIN usCitiesNorm as usc
    ON ps.state_id = usc.state_id and ps.city = usc.city
    JOIN caUE as ue 
    ON ps.state_id = ue.state_id and usc.county = ue.county
    WHERE usc.state_name = 'California'
""").show(20)

+----------+--------+--------------------+------------+----+------+--------------+------------+-----+-----------+--------------+------------------+-----------------+--------------------+------------+
|state_name|  county|                name|       armed| age|gender|mental_illness|threat_level| flee|body_camera|          race|unemployment_count|unemployment_rate|         agency_name| agency_dept|
+----------+--------+--------------------+------------+----+------+--------------+------------+-----+-----------+--------------+------------------+-----------------+--------------------+------------+
|California|   Kings|        Dave Phoenix|     replica|  25|  male|         false|       point|  not|      false|        Native|              5682|              9.7|Lemoore Police De...|local_police|
|California|   Kings|Sergio Velasco-Ma...|blunt_object|  25|  male|         false|      attack|  not|      false|      Hispanic|              5682|              9.7|Avenal Police Dep...|local_police|


In [10]:
spark.stop()